In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots 
import plotly.graph_objs as go 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix,f1_score,accuracy_score, precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")
submit=pd.read_csv("upload.csv")
test_set['Exited'] = 0
data = train_set.append(test_set) #先把訓練集和測試集合併，較好處理資料
data.reset_index(inplace=True, drop=True)
data = data.drop("CustomerId",axis = 1)
data = data.drop("Surname",axis = 1)
data

FileNotFoundError: [Errno 2] No such file or directory: 'upload.csv'

In [108]:
import category_encoders as ce
encoder = ce.LeaveOneOutEncoder(sigma = 0.05)
data['Geography'] = encoder.fit_transform(data['Geography'], data['Exited'])
data['Gender'] = encoder.fit_transform(data['Gender'], data['Exited'])
data['Tenure'] = encoder.fit_transform(data['Tenure'], data['Exited'])
data['NumOfProducts']=data['NumOfProducts'].replace({1:4039,2:3697,3:215,4:49})
data['HasCrCard']=data['HasCrCard'].replace({1:5622,0:5378})
data['IsActiveMember']=data['IsActiveMember'].replace({1:4135,0:3865})

data["Balance"] = (data["Balance"] - data["Balance"].min()) / (data["Balance"].max()-data["Balance"].min())
data["EstimatedSalary"] = (data["EstimatedSalary"] - data["EstimatedSalary"].min()) / (data["EstimatedSalary"].max()-data["EstimatedSalary"].min())
data["CreditScore"] = (data["CreditScore"] - data["CreditScore"].min()) / (data["CreditScore"].max()-data["CreditScore"].min())
data["NumOfProducts"] = (data["NumOfProducts"] - data["NumOfProducts"].min()) / (data["NumOfProducts"].max()-data["NumOfProducts"].min())
data

,RowNumber,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,551,0.892508,0.124499,0.130458,38,5,0.409621,0.914286,5378,4135,0.589163,0
1,6897,0.768730,0.129884,0.189399,54,4,0.000000,1.000000,5622,3865,0.654032,1
2,4588,0.736156,0.116372,0.202471,31,5,0.456025,1.000000,5622,4135,0.887514,0
3,291,0.475570,0.136698,0.196458,40,4,0.331954,1.000000,5622,3865,0.627770,0
4,1673,1.000000,0.125283,0.141336,42,5,0.459864,1.000000,5378,4135,0.810459,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7872,0.592834,0.125014,0.135442,35,6,0.000000,0.914286,5622,4135,0.455262,0
9996,4257,0.628664,0.244462,0.128358,41,5,0.283552,0.914286,5622,3865,0.612166,0
9997,2273,0.332248,0.119943,0.199960,46,3,0.449480,0.914286,5622,4135,0.696842,0
9998,315,0.508143,0.136467,0.206752,44,7,0.000000,0.914286,5622,4135,0.831143,0


In [109]:
train_data = data.iloc[:8000,:] #將訓練集和測試集分開來
test_data = data.iloc[8000:,:]
train_data.drop("RowNumber",axis = 1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.892508,0.124499,0.130458,38,5,0.409621,0.914286,5378,4135,0.589163,0
1,0.768730,0.129884,0.189399,54,4,0.000000,1.000000,5622,3865,0.654032,1
2,0.736156,0.116372,0.202471,31,5,0.456025,1.000000,5622,4135,0.887514,0
3,0.475570,0.136698,0.196458,40,4,0.331954,1.000000,5622,3865,0.627770,0
4,1.000000,0.125283,0.141336,42,5,0.459864,1.000000,5378,4135,0.810459,0
...,...,...,...,...,...,...,...,...,...,...,...
7995,0.397394,0.129636,0.183637,35,6,0.465350,0.914286,5622,4135,0.792944,1
7996,0.508143,0.239007,0.187934,45,7,0.661749,1.000000,5622,3865,0.491455,0
7997,0.758958,0.137732,0.192232,43,5,0.558373,1.000000,5622,3865,0.862275,1
7998,0.876221,0.129772,0.141663,38,4,0.446714,1.000000,5378,3865,0.446828,0


In [110]:
X = train_data.drop(['Exited'], axis=1)
y = train_data['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
gradient_boosting = GradientBoostingClassifier(n_estimators=50,max_depth= 13, min_samples_split= 1000,min_samples_leaf= 7,max_features=11,subsample=0.85)
gradient_boosting.fit(X_train, y_train)
y_pred_gradient_boosting = gradient_boosting.predict(X_test)
accuracy_score(y_test, y_pred_gradient_boosting)

0.865625

In [94]:
param_test1 = {'n_estimators':np.arange(10,101,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=50), 
param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
gsearch1.fit(X,y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([0.12355866, 0.22243686, 0.38835168, 0.68330727, 0.68254399,
         0.56884217, 0.67180891, 0.77225361, 0.82893295, 0.91301541]),
  'std_fit_time': array([0.00519347, 0.00809607, 0.08266385, 0.06302119, 0.15415232,
         0.02716282, 0.02656955, 0.03208407, 0.00895215, 0.01139152]),
  'mean_score_time': array([0.00580149, 0.00699759, 0.01120019, 0.01380272, 0.00999918,
         0.01018105, 0.01040158, 0.00980344, 0.00980062, 0.00919766]),
  'std_score_time': array([0.00116623, 0.00209481, 0.00503448, 0.0095782 , 0.00389795,
         0.00094696, 0.00149625, 0.00172152, 0.00074763, 0.00159852]),
  'param_n_estimators': masked_array(data=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
               mask=[False, False, False, False, False, False, False, False,
                     False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 10},
   {'n_estimators': 20},
   {'n_estimators': 30},
   {'n_estimators': 40},
   {'n_e

In [96]:
param_test2 = {'max_depth':range(5,15,2), 'min_samples_split':range(200,1001,100)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50,max_features='sqrt', subsample=0.8, random_state=50), 
param_grid = param_test2, scoring='roc_auc',n_jobs=4, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

({'mean_fit_time': array([0.43208485, 0.34602299, 0.3398416 , 0.34345765, 0.30177412,
         0.30765371, 0.29286714, 0.32883964, 0.3261291 , 0.41234379,
         0.38796077, 0.38089862, 0.35120082, 0.3446939 , 0.36247125,
         0.35422139, 0.32843089, 0.31383362, 0.4846467 , 0.44445744,
         0.41863961, 0.45922427, 0.41867027, 0.44600782, 0.49297781,
         0.50121984, 0.38362565, 0.56115375, 0.57176023, 0.51819005,
         0.48231344, 0.97852864, 1.04552712, 0.94867725, 0.93930488,
         0.92544851, 1.67952523, 1.48338084, 0.75331678, 0.42650537,
         0.41028132, 0.39004211, 0.36272812, 0.3957026 , 0.31419244]),
  'std_fit_time': array([0.03019988, 0.0116925 , 0.01153767, 0.01799604, 0.00954835,
         0.00812942, 0.00530385, 0.02620706, 0.0268843 , 0.01012915,
         0.01040663, 0.01369862, 0.00386921, 0.01511137, 0.01649438,
         0.00599052, 0.00819556, 0.00844912, 0.01536472, 0.00613821,
         0.0145564 , 0.02991006, 0.01761371, 0.02838984, 0.02611853,

In [97]:
param_test3 = {'min_samples_split':np.arange(1000,2100,200), 'min_samples_leaf':np.arange(30,71,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=1000,n_estimators=50,max_depth=7,max_features='sqrt',subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='roc_auc',n_jobs=4, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

({'mean_fit_time': array([0.30769844, 0.31116381, 0.32495189, 0.28300729, 0.21530147,
         0.19678545, 0.29118333, 0.27534065, 0.30286994, 0.25050073,
         0.21828952, 0.20292363, 0.28119531, 0.26977253, 0.24256601,
         0.22553372, 0.21410866, 0.2098897 , 0.28470016, 0.26440144,
         0.27725554, 0.26088953, 0.21700301, 0.19542561, 0.28764987,
         0.26222672, 0.23933887, 0.22379375, 0.21013322, 0.18725796]),
  'std_fit_time': array([0.00978483, 0.02387011, 0.01789403, 0.02082408, 0.00691007,
         0.00653516, 0.00896908, 0.01370941, 0.01936159, 0.01017579,
         0.01015963, 0.01079677, 0.01059288, 0.00933855, 0.00495646,
         0.00869637, 0.00819282, 0.00952758, 0.00268836, 0.00902409,
         0.00985732, 0.01095048, 0.00525033, 0.0125435 , 0.00511928,
         0.00981573, 0.00893201, 0.00946619, 0.00915971, 0.00630929]),
  'mean_score_time': array([0.00825233, 0.0081965 , 0.00740786, 0.00689416, 0.00699916,
         0.00560017, 0.00639949, 0.0061996 , 0.

In [102]:
param_test4 = {'min_samples_leaf':np.arange(7,19,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=1000,min_samples_leaf=9, n_estimators=50,max_depth=7,max_features=8, subsample=0.8, random_state=10), 
param_grid = param_test4, scoring='roc_auc',n_jobs=4, cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

({'mean_fit_time': array([0.61480613, 0.56717167, 0.58325839, 0.54053011, 0.56268334,
         0.56831269]),
  'std_fit_time': array([0.01672056, 0.01833028, 0.0126273 , 0.02320527, 0.01082495,
         0.02370846]),
  'mean_score_time': array([0.00519924, 0.0074646 , 0.0061882 , 0.00682039, 0.00724683,
         0.00580115]),
  'std_score_time': array([0.00075068, 0.00100225, 0.00076263, 0.00172313, 0.00200624,
         0.00160205]),
  'param_min_samples_leaf': masked_array(data=[7, 9, 11, 13, 15, 17],
               mask=[False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'min_samples_leaf': 7},
   {'min_samples_leaf': 9},
   {'min_samples_leaf': 11},
   {'min_samples_leaf': 13},
   {'min_samples_leaf': 15},
   {'min_samples_leaf': 17}],
  'split0_test_score': array([0.8506184 , 0.85034879, 0.85141968, 0.85123492, 0.84947775,
         0.84999811]),
  'split1_test_score': array([0.85561086, 0.85710596, 0.85510558, 0.85457391, 

In [100]:
param_test5 = {'max_features':np.arange(8,15,2)}
gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=1000,min_samples_leaf=11, n_estimators=50,max_depth=7,max_features=8, subsample=0.8, random_state=10), 
param_grid = param_test5, scoring='roc_auc',n_jobs=4, cv=5)
gsearch5.fit(X_train,y_train)
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

c:\Users\90607\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\90607\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\90607\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_gb.py", line 586, in fit
    n_stages = self._fit_stages(
  File "c:\Users\90607\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ense

({'mean_fit_time': array([0.65630679, 0.71471515, 0.00646429, 0.00768366]),
  'std_fit_time': array([0.04994246, 0.040183  , 0.00077648, 0.00144954]),
  'mean_score_time': array([0.00641685, 0.00611248, 0.        , 0.        ]),
  'std_score_time': array([0.00160959, 0.00134911, 0.        , 0.        ]),
  'param_max_features': masked_array(data=[8, 10, 12, 14],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_features': 8},
   {'max_features': 10},
   {'max_features': 12},
   {'max_features': 14}],
  'split0_test_score': array([0.85141968, 0.85301471,        nan,        nan]),
  'split1_test_score': array([0.85510558, 0.85474925,        nan,        nan]),
  'split2_test_score': array([0.86065234, 0.8628733 ,        nan,        nan]),
  'split3_test_score': array([0.87273602, 0.8696378 ,        nan,        nan]),
  'split4_test_score': array([0.87494125, 0.87258562,        nan,        nan]),
  'mean_test_score': 

In [104]:
param_test6 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch6 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=1000,min_samples_leaf=9, n_estimators=50,max_depth=7,max_features=8, subsample=0.9, random_state=10), 
param_grid = param_test6, scoring='roc_auc',n_jobs=4, cv=5)
gsearch6.fit(X_train,y_train)
gsearch6.cv_results_, gsearch6.best_params_, gsearch6.best_score_

({'mean_fit_time': array([0.60960946, 0.72658491, 0.74367743, 0.80912519, 0.61579895,
         0.67607613]),
  'std_fit_time': array([0.07562747, 0.13941898, 0.13795904, 0.13052413, 0.03372379,
         0.02813912]),
  'mean_score_time': array([0.00627403, 0.0065959 , 0.00819788, 0.0067996 , 0.00619698,
         0.00480189]),
  'std_score_time': array([0.001055  , 0.0017448 , 0.0020419 , 0.00239529, 0.00097909,
         0.00117164]),
  'param_subsample': masked_array(data=[0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
               mask=[False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'subsample': 0.6},
   {'subsample': 0.7},
   {'subsample': 0.75},
   {'subsample': 0.8},
   {'subsample': 0.85},
   {'subsample': 0.9}],
  'split0_test_score': array([0.84970965, 0.85215309, 0.85059201, 0.85034879, 0.85202866,
         0.85526207]),
  'split1_test_score': array([0.85634615, 0.85562594, 0.85328243, 0.85710596, 0.85784879,
         0.858316

In [111]:
X_train= train_data.drop(['Exited'], axis=1)
y_train= train_data['Exited']
X_test=test_data.drop(['Exited'], axis=1)
y_test=test_data['Exited']

gradient_boosting = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=1000,min_samples_leaf=9, n_estimators=50,max_depth=7,max_features=8, subsample=0.9, random_state=10)
gradient_boosting.fit(X_train, y_train)
y_pred_gradient_boosting = gradient_boosting.predict(X_test)
submit = pd.DataFrame({
        "RowNumber": test_data["RowNumber"],
        "Exited": y_pred_gradient_boosting
    })
submit.to_csv(r"C:\Users\90607\OneDrive\桌面\upload.csv", index=False)
submit['Exited'].value_counts()

0    1713
1     287
Name: Exited, dtype: int64

在沒有離開的人當中，男生佔多數；在離開的人當中，女生佔多數